In [1]:
push!(LOAD_PATH, pwd())

using PyPlot, Statistics, CPUTime
using SpinBodies, MBTrees;

┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1260
┌ Warning: Module VersionParsing with build ID 47460475475961 is missing from the cache.
│ This may mean VersionParsing [81def892-9a0e-5fdd-b105-ffc91e053289] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1016
┌ Warning: Module VersionParsing with build ID 47460475475961 is missing from the cache.
│ This may mean VersionParsing [81def892-9a0e-5fdd-b105-ffc91e053289] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1016
┌ Info: Skipping precompilation since __precompile__(false). Importing PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee].
└ @ Base loading.jl:1033
┌ Info: Precompiling PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1260
┌ Warning: Module VersionParsing with build ID 47460475475961 is missing from the cache.
│ This may mean VersionParsing [81def892-9a0e-5fdd-b105-ffc91e053289] does

LoadError: InitError: PyError (PyImport_ImportModule

The Python package matplotlib could not be found by pyimport. Usually this means
that you did not install matplotlib in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/usr/bin/python3

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the matplotlib module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg.build("Py

In [2]:
const version = :tri
const steps = 500000
const N = 10

const Ts = 0.1:0.1:4;

In [3]:
freeman_data = []

for T in Ts
    lattice = SpinLattice(N, T)
    tree = build_tree(lattice, version)
    times = []
    for i = 1:25
        time = @CPUelapsed for step = 1:steps
            freeman_step!(lattice, tree)
        end
        push!(times, time)
    end
    push!(freeman_data, mean(times))
end

LoadError: UndefVarError: @CPUelapsed not defined

In [4]:
freeman_fast_data = []

for T in Ts
    lattice = SpinLattice(N, T)
    tree = build_tree(lattice, version)
    times = []
    for i = 1:25
        time = @CPUelapsed for step = 1:steps
            freeman_fast_step!(lattice, tree)
        end
        push!(times, time)
    end
    push!(freeman_fast_data, mean(times))
end

UndefVarError: UndefVarError: freeman_fast_step! not defined

In [5]:
metropolis_data = []
metropolis_time_per_flip = []

for T in Ts
    lattice = SpinLattice(N, T)
    times = []
    flips = []
    for i = 1:25
        time = @CPUelapsed for step = 1:steps
            metropolis_step!(lattice)
        end
        push!(times, time)
        push!(flips, lattice.flips)
    end
    push!(metropolis_data, mean(times))
    push!(metropolis_time_per_flip, mean(times) / mean(flips))
end

InterruptException: InterruptException:

In [8]:
semilogy(Ts, freeman_data ./ metropolis_data)
semilogy(Ts, freeman_fast_data ./ metropolis_data)
legend(["freeman_CPUtime / metropolis_CPUtime", "freeman_fast_CPUtime / metropolis_CPUtime"])
# savefig("freeman_CPUtime_per_metropolis_CPUtime.pdf")

DimensionMismatch: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 12 and 0")

In [9]:
semilogy(Ts, metropolis_time_per_flip ./ metropolis_time_per_flip[end])
legend(["metropolis_CPUtime / flip"])
# savefig("metropolis_CPUtime_per_flip.pdf")

BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index [0]